In [ ]:
# TODO
# Group leagues according to membership - COMPLETE
# Derive draft results for each league (different handling for snake vs auction draft)
# Get weekly player results data

#APIs to try
#https://github.com/bbenbenek/nfl-fantasy-football
#https://github.com/uberfastman/yfpy
#https://github.com/whatadewitt/yahoo-fantasy-sports-api
#https://pypi.org/project/nflgame/
#https://github.com/fantasydatapros/nflfastpy

In [ ]:
import yahoo_fantasy_api as yfa
from yahoo_oauth import OAuth2
import pandas as pd
import json

'''
creds = {'consumer_key': 'dj0yJmk9ZU80dWc0ZWxzZTQ4JmQ9WVdrOVdsWTBRMkUxTUVZbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PWIw',
         'consumer_secret': 'c2a9bd5a9ec8a11e6eb326aae044bdc6ee245972'}
with open(args['<json>'], "w") as f:
    f.write(json.dumps(creds))
oauth = OAuth2(None, None, from_file='oauth2.json')
'''

#this oauth file must have been generated by me, not sure the instructions to do so.  probably easy.
oauth = OAuth2(None, None, from_file='oauth2.json')

#API docs: https://yahoo-fantasy-api.readthedocs.io/en/latest/yahoo_fantasy_api.html

football = yfa.game.Game(oauth, 'nfl')

In [ ]:
leagues = football.league_ids()
    
league_dfs = []
for league in leagues:
    league_settings=football.to_league(league).settings()
    league_dfs.append(pd.DataFrame(league_settings))
leagues_df = pd.concat(league_dfs)
leagues_df.reset_index(drop=True,inplace=True)
leagues_df

In [ ]:
#assemble managers for each league
#this cell takes a few minutes to run
managers = []
for league in leagues:
    lg = football.to_league(league)
    for k in lg.teams().keys():
        teams = lg.teams()[k]
        manager = lg.teams()[k]['managers'][0]['manager']
        managers.append((k,lg.settings()['season'],lg.settings()['name'],teams['name'],manager['nickname']))
managers

In [ ]:
#group leagues together if manager membership meets threshold
match_threshold = .75

df = pd.DataFrame(managers,columns=['league_key','season','league_name','team','manager'])
league_keys = []
for key in df.league_key:
    league_keys.append('.'.join(key.split('.')[:-2]))
df['league_key'] = league_keys

df.sort_values(by=['season','league_name'],ascending=False,inplace=True)
seasons_df = df[['league_key','season','league_name']].copy()
seasons_df.drop_duplicates(subset=['season','league_name'],inplace=True) #could use season_id for this?
seasons = []
for index, row in seasons_df.iterrows():
    seasons.append((row['season'],row['league_name'],row['league_key']))
league_profiles = {}
league_mappings = []
i=0
for s in seasons:
    matched = False
    league_df = df[(df.season == s[0]) & (df.league_name == s[1])]
    if len(league_profiles)==0:
        historical_managers = []
        historical_teams = []
        for index, row in league_df.iterrows():
            if row['manager'] != '-- hidden --':
                historical_managers.append(row['manager'])
            if row['team'] != '-- hidden --':
                historical_teams.append(row['team'])
        league_profiles[i] = (league_df,historical_managers,historical_teams) #creating the first profile which is the most recent season 
        league_mappings.append((s[2],i))
        i=i+1
    else:
        for l in league_profiles.keys(): #compare current league to set of already identified league classes
            if matched == False:
                profile_managers = league_profiles[l][1].copy()
                profile_teams = league_profiles[l][2].copy()
                match_score = 0
                for index, row in league_df.iterrows():
                    if row['manager'] in profile_managers:
                        match_score = match_score + 1
                    elif row['team'] in profile_teams:
                        match_score = match_score + 1
                    #updating profile lists in case we need to update the dictionary
                    if (row['manager'] != '-- hidden --') & (row['manager'] not in profile_managers):
                        profile_managers.append(row['manager'])
                    if (row['team'] != '-- hidden --') & (row['team'] not in profile_teams):
                        profile_teams.append(row['team'])
                if match_score / league_df.shape[0] > match_threshold:
                    league_mappings.append((s[2], l))
                    league_profiles[l] = (league_profiles[l][0],profile_managers,profile_teams) #update dictionary with latest names
                    matched = True
        #we assume there were no matches, so create a new profile
        if matched == False:
            league_profiles[i] = (league_df, list(league_df['manager']), list(league_df['team']))
            league_mappings.append((s[2],i))
            i=i+1

season_dict = {}
for m in league_mappings:
    season_dict[m[0]] = m[1]

league_ids = []
for league_key in leagues_df.league_key:
    league_ids.append(season_dict[league_key])
leagues_df['group_id'] = league_ids

leagues_df.sort_values(by=['group_id','season'],ascending=[True,False],inplace=True)
leagues_df.drop_duplicates(subset=['season','group_id'])[['name','season','group_id']]

In [ ]:
from datetime import datetime
datetime.date(2021,9,12)

In [ ]:
datetime(2021, 9, 12).isocalendar().week

In [ ]:
import datetime
pre_recent = football.to_league('371.l.347949')
day = datetime.date(2021, 9, 12) # can select any random day the player has played
player=[3983] # can select any player
pre_recent.player_stats(player_ids=[5228,32671,8432],req_type='date',date=day)

In [ ]:
pre_recent.positions()

In [ ]:
#helpful https://github.com/whatadewitt/yahoo-fantasy-sports-api/issues/79
pre_recent.player_stats(player_ids=[5228,32671,8432],req_type='date',date=36)

In [ ]:
from pathlib import Path
from yfpy.query import YahooFantasySportsQuery
auth = 'C:/Users/dunca/AppData/Local/Programs/Python/Python310/Lib/site-packages/yfpy'
query = YahooFantasySportsQuery(Path(auth), league_id="696126")
query.get_league_matchups_by_week(1)

In [ ]:
pre_recent.player_stats(player_ids=[5228,32671,8432],req_type='season',season=2021)

In [ ]:
leagues

In [ ]:
lg.player_stats(32671,'date',date=None)#datetime.date(2021,12,24))

In [ ]:
yahoo_public = football.to_league('406.l.1273347')
yahoo_public.settings()

In [ ]:
yahoo_public.draft_results()

In [ ]:
lg.matchups()

In [ ]:
recent_league.standings()

In [ ]:
leagues

In [ ]:
recent_league.taken_players()

In [ ]:
#my own key
recent_league.team_key()

In [ ]:
recent_league.transactions('add','')#add,drop,commish,trade
#losing faab_bid are not available publicly :(


In [ ]:
yahoo_public.teams()[yahoo_public.team_key()]['managers'][0]['manager']

In [ ]:
yahoo_public.teams()[yahoo_public.team_key()]['managers']

In [ ]:
yahoo_public.teams()

In [ ]:
my_team = yahoo_public.to_team(yahoo_public.team_key())

In [ ]:
#need to handle with precision what players have actually started
#though....if you start a player on thursday then presumably they are still in that slot on sunday/monday
#complexity is with games that are on different days and you could make a chnage to lineup at the last minute
my_team.roster(week=1)

In [ ]:
#trying yfpy
# https://yfpy.uberfastman.com/_autosummary/yfpy.query.YahooFantasySportsQuery.html
from pathlib import Path
from yfpy.query import YahooFantasySportsQuery
auth = 'C:/Users/dunca/AppData/Local/Programs/Python/Python310/Lib/site-packages/yfpy'
query = YahooFantasySportsQuery(Path(auth), league_id="696126")

#PRE 348.l.216667
#yahoo public 406.l.1273347
#Sexiest 399.l.522487
#query.get_league_draft_results() #does not work PRE
#query.get_league_info() # does not work for PRE
query.get_league_standings() # does not work for PRE
#query.get_league_key()

In [ ]:
#trying nfl-fantasy-football
# https://github.com/bbenbenek/nfl-fantasy-football
# need to follow the instructions



In [ ]:
#trying nflfastpy
# https://github.com/fantasydatapros/nflfastpy
# probably not relevant, this is nfl data not fantasy specific data
import nflfastpy

df = nflfastpy.load_pbp_data(year=2021)
roster_df = nflfastpy.load_roster_data(year=2019)
roster_df